In [1]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import seaborn as sns
import FinancialMachineLearning as fml

In [2]:
from random import gauss
from itertools import product

def main() :
    rPT = rSLm = np.linspace(0, 10, 21)
    count = 0
    for prod_ in product([10, 5, 0, -5, -10], [5, 10, 25, 50, 100]):
        count += 1
        coeffs = {'forecast' : prod_[0], 'hl' : prod_[1], 'sigma' : 1}
        output = batch(coeffs, nIter = 1e5, maxHP = 100, rPT = rPT, rSLm = rSLm)
    return output

def batch(coeffs, nIter = 1e5, maxHP = 100, rPT = np.linspace(.5, 10, 20), rSLm = np.linspace(.5, 10, 20), seed = 42) :
    phi, output1 = 2 ** (-1 / coeffs['hl']), []
    for comb_ in product(rPT, rSLm) :
        output2 = []
        for iter_ in range(int(nIter)) :
            p, hp, count = seed, 0, 0
            while True :
                p = (1 - phi) * coeffs['forecast'] + phi * p + coeffs['sigma'] * gauss(0,1)
                cP = p - seed
                hp += 1
                if cP > comb_[0] or cP < -comb_[1] or hp > maxHP :
                    output2.append(cP)
                    break
        mean, std = np.mean(output2), np.std(output2)
        print(comb_[0], comb_[1], mean, std, mean/std)
        output1.append((comb_[0], comb_[1], mean, std, mean/std))
    return output1

### Chapter 13. Synthetic Data Backtest

#### Exercise 1
당신이 거래 실행 트레이더라고 가정해 보자. 고객이 전화를 걸어 가격 100에 진입한 매도 포지션을 커버하기 위한 주문을 요청했다. 고객은 두 가지 청산 조건을 제시했다. 즉, 105에서의 이익 실현과 90에서의 손절이다.

**(a)** 고객이 가격은 O-U프로세스를 따른다고 믿는다고 가정하면 이 수준이 적절한가? 어떤 파라미터에서 적절한가?